In [1]:
# Loading my orofacial module
import sys
sys.path.append("/Users/eric/Workspace/wangLab/orofacial")
import orofacial

# Load the h5 file of interest
h5_path = "data/phox2b38_20240321_1_tongue.h5"
h5 = orofacial.tongue_mask_processing.TongueArchive(h5_path)
display(f"Frame count: {len(h5.frames)}")

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


'Frame count: 34286'

In [ ]:
import tqdm
from matplotlib import pyplot as plt

# Extract images from the archive and pair them with their true frame number
imgs = []
for i in tqdm.tqdm(range(len(h5.frames))):
    imgs.append((orofacial.tongue_mask_processing.plot_frame_bool(h5, i, 256, 256), h5.frames[i]))

plt.imshow(imgs[1000][0], cmap='gray')

In [ ]:
import numpy as np

# removing images with less than 15 pixels as preprocessing step
imgs = list(filter(lambda x : np.count_nonzero(x[0]) > 15, tqdm.tqdm(imgs)))

In [ ]:
# keep only the largest connected component of each image
imgs = list(map(lambda x : (orofacial.tongue_mask_processing.keep_largest_cc(x[0].astype(np.uint8)).astype(np.bool_), x[1]), tqdm.tqdm(imgs)))

In [ ]:
# Extract the tongue tip coordinates from the images 
coords = list(map(lambda x : orofacial.tongue_tip_track_2D.find_tongue_tip(x[0], np.array([-1, 1], dtype=np.float32)), tqdm.tqdm(imgs)))

In [ ]:
from matplotlib.patches import Rectangle

feed_tube_top_left = (0, 290)
feed_tube_bottom_right = (239, 346)
def plot_feed_tube():
    plt.gca().add_patch(Rectangle(feed_tube_top_left, feed_tube_bottom_right[0]-feed_tube_bottom_right[0], feed_tube_top_left[1]-feed_tube_bottom_right[1], linewidth=1, edgecolor="#00FF00", facecolor='none', zorder=10))

zoom_xlim = [150, 400]
zoom_ylim = [450, 150]
def set_zoom():
    plt.gca().set_xlim(zoom_xlim)
    plt.gca().set_ylim(zoom_ylim)
    plt.gca().set_box_aspect(1)

plt.imshow(imgs[1000][0], extent=[0, 640, 480, 0], cmap="gray")
plot_feed_tube()

In [ ]:
# first look at tracking result
# analysis operates in 256x256 so for display purpose they are scaled to the proper 640x480 dimension
x, y = np.column_stack(coords)
plt.gca().invert_yaxis()
plt.scatter(x*(640/256), y*(480/256), s=1)
plot_feed_tube()

In [ ]:
# define a lick to be a contiguous region of tracked frames where tongue is visible
# the archive only contains frames where the tongue is visible, so we are looking for gaps in frame number
licks = [[coords[0]]]
for i in range(1, len(coords)):
    if isinstance(coords[i], np.ndarray):
        if imgs[i][1] != imgs[i-1][1] + 1:
            licks.append([coords[i]])
        else:
            licks[-1].append(coords[i])
display(f"{len(licks)} licks")

In [ ]:
# Showing the licks with some transparency
plt.gca().invert_yaxis()
for p in licks:
    x, y = np.column_stack(p)
    plt.plot(x*(640/256), y*(480/256), alpha=0.05, c="blue")
plot_feed_tube()
set_zoom()

In [ ]:
import scipy

# Interpolating the licks, returns 100 points along interpolation effectively parameterizing the lick
def interp(p):
    tck, u = scipy.interpolate.splprep(np.column_stack(p), s=0, k=3) 
    t = np.linspace(0, 1, 100)
    x_new, y_new = scipy.interpolate.splev(t, tck)
    return list(zip(x_new, y_new))

# Applying interpolation on all licks
licks_p = []
for i in range(len(licks)):
    # splprep requires at least 3 points and no duplicate points, preprocessing is needed
    p = [licks[i][0]]
    for j in range(1, len(licks[i])):
        if np.linalg.norm(np.array(licks[i][j]) - np.array(p[-1])) > 0.001:
            p.append(licks[i][j])
    if len(p) > 3:
        licks_p.append(interp(p))

In [ ]:
# Plotting the interpolated licks, shouldn't be much of a difference
plt.gca().invert_yaxis()
for p in licks_p:
    x, y = np.column_stack(p)
    plt.plot(x*(640/256), y*(480/256), alpha=0.05, c="blue")
plot_feed_tube()
set_zoom()